<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/Google_Earth_Engine_Sentinel_1_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentinel-1은 유럽연합(EU: European Union)이 자금을 지원하고 유럽우주국(ESA: European Space Agency)이 코페르니쿠스 프로그램(Copernicus Programme) 내에서 수행하는 우주 미션입니다. Sentinel-1은 다양한 편파와 해상도에서 C-밴드 합성 개구 레이더(SAR: Synthetic Aperture Radar) 이미지를 수집합니다. 레이더 데이터는 보정된, 정사영상 이미지를 얻기 위해 여러 전문 알고리즘이 필요하기 때문에 Earth Engine에서 Sentinel-1 데이터의 사전 처리를 설명합니다.

Sentinel-1 데이터는 다양한 기기 구성, 해상도, 대역 조합으로 상승 궤도와 하강 궤도 모두에서 수집됩니다. 이러한 이질성 때문에 처리를 시작하기 전에 데이터를 동질의 부분집합으로 필터링하는 것이 보통 필요합니다.

In [2]:
import ee
import geemap
import geopandas as gpd
import requests

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

Sentinel-1 데이터의 동질 부분집합을 만들기 위해서는 보통 메타데이터 속성을 사용하여 컬렉션을 필터링할 필요가 있습니다. 필터링에 사용되는 일반적인 메타데이터 필드에는 다음과 같은 속성들이 포함됩니다:

1. `transmitterReceiverPolarisation`: ['VV'], ['HH'], ['VV', 'VH'], 또는 ['HH', 'HV']
2. `instrumentMode`: 'IW' (Interferometric Wide Swath), 'EW' (Extra Wide Swath) 또는 'SM' (Strip Map).
3. `orbitProperties_pass`: 'ASCENDING' 또는 'DESCENDING'
4. `resolution_meters`: 10, 25 또는 40
5. `resolution`: 'M' (medium) 또는 'H' (high).

다음 코드는 `transmitterReceiverPolarisation`, `instrumentMode`, 및 `orbitProperties_pass` 속성에 따라 Sentinel-1 컬렉션을 필터링하고, 여러 관측 조합에 대한 복합 이미지를 계산한 후 이러한 특성이 데이터에 어떻게 영향을 미치는지 보여주기 위해 지도에 표시합니다.

In [3]:
# Sentinel-1 이미지 컬렉션을 불러오고, 2023년 6-9월까지의 관측 데이터로 필터링
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(
    '2023-06-01', '2023-10-01'
)

In [4]:
# 메타데이터 속성에 따라 Sentinel-1 컬렉션 필터링
vv_vh_iw = (
    sentinel_1
    # VV와 VH 이중 편파로 촬영된 이미지를 얻기 위해 필터링
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    # IW 모드로 수집된 이미지를 얻기 위해 필터링
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
)

In [5]:
# 상승 궤도와 하강 궤도 이미지를 별도의 컬렉션으로 분리
vv_vh_iw_asc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
)
vv_vh_iw_desc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
)

In [6]:
# 시각화를 위해 다양한 관측의 시간 평균 계산
# 상승 궤도 VH 평균
vh_iw_asc_mean = vv_vh_iw_asc.select('VH').mean()
# 하강 궤도 VH 평균
vh_iw_desc_mean = vv_vh_iw_desc.select('VH').mean()
# 상승 및 하강 궤도 이미지 컬렉션을 결합한 VV 평균.
vv_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VV').mean()
# 상승 및 하강 궤도 이미지 컬렉션을 결합한 VH 평균.
vh_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VH').mean()

  > `"wb"`는 'write binary'의 줄임말로, 파일을 쓰기 위해 바이너리 모드를 사용하겠다는 의미입니다. 파일은 크게 텍스트 모드와 바이너리 모드로 두 가지 방식으로 다룰 수 있으며, 바이너리 모드는 파일을 파이트 형태로, 즉 파일의 원래 형식과 정확한 내용을 변경 없이 저장하는 방식을 말합니다.

In [8]:
# 진주시 센서스경계 다운로드
url = "https://github.com/osgeokr/GEE-PAM-Book/raw/main/JINJU.gpkg"
response = requests.get(url)
with open("JINJU.gpkg", "wb") as file:
    file.write(response.content)

# 진주시 센서스경계 읽기
gdf = gpd.read_file("JINJU.gpkg")
gdf.head()

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry
0,20230701,진주시,38030,"MULTIPOLYGON (((1059659.398 1706201.131, 10596..."


In [12]:
# 좌표계를 EPSG:4326으로 변환
gdf = gdf.to_crs(epsg=4326)

# 중앙점 계산
gdf['centroid'] = gdf.geometry.centroid

# 중앙점의 첫 번째 요소의 좌표 출력
centroid_point = gdf['centroid'].iloc[0]
longitude, latitude = centroid_point.x, centroid_point.y
print("Longitude:", longitude, "Latitude:", latitude)

Longitude: 128.12976291238812 Latitude: 35.20514831522156


<ipython-input-12-2edad1727a83>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [14]:
# 다양한 관측의 시간 평균을 표시 및 비교
m = geemap.Map()
m.add_layer(vv_iw_asc_desc_mean, {'min': -12, 'max': -4}, 'vv_iw_asc_desc_mean')
m.add_layer(vh_iw_asc_desc_mean, {'min': -18, 'max': -10}, 'vh_iw_asc_desc_mean')
m.add_layer(vh_iw_asc_mean, {'min': -18, 'max': -10}, 'vh_iw_asc_mean')
m.add_layer(vh_iw_desc_mean, {'min': -18, 'max': -10}, 'vh_iw_desc_mean')
m.set_center(-73.8719, 4.512, 9)  # 보고타, 콜롬비아
m

Map(center=[4.512, -73.8719], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [15]:
m = geemap.Map(center=[35.3236, 127.6366],
                 zoom=11,
                 layout={'height':'400px', 'width':'800px'})
m

Map(center=[35.3236, 127.6366], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…